In [ ]:
from QLBM import QLBM, collision, InitializeQC
import numpy as np
import matplotlib.pyplot as plt
import qiskit_aer
from qiskit import transpile

In [ ]:
# Grid size
N_POINTS_X, N_POINTS_Y, N_POINTS_Z = 16, 16, 16

# Simulation parameters
TIMESTEPS = 5
c_s = 1 / np.sqrt(3)
NUMBER_DISCRETE_VELOCITIES = 27  # D2Q27

# Diffusion coefficient
diffusion_coefficient = 0.3 * c_s**2

In [ ]:
Psi_init = np.zeros((N_POINTS_X, N_POINTS_Y, N_POINTS_Z))
Psi_init=np.full([N_POINTS_X,N_POINTS_Y,N_POINTS_Z],0.1)
Psi_init[:,int(N_POINTS_X/2)-1,:]=0.3
Psi_init[:,:,int(N_POINTS_X/2-1)]=0.3
Psi_init[int(N_POINTS_X/2-1),:,:]=0.3

In [ ]:
# Parameters for velocity field
A, B, C = 0.2, 0.2, 0.2  # Amplitude of velocity components
a, b, c = 1, 1, 1    # Frequency parameters

# Initialize velocity field
u = np.zeros((N_POINTS_X, N_POINTS_Y, N_POINTS_Z, 3))
x=np.linspace(0,2*np.pi,N_POINTS_X)
y=np.linspace(0,2*np.pi,N_POINTS_Y)
z=np.linspace(0,2*np.pi,N_POINTS_Z)
# Populate the velocity field
for i in range(N_POINTS_X):
    for j in range(N_POINTS_Y):
        for k in range(N_POINTS_Z):
            u[i, j, k, 0] = A * np.cos(a * x[i]) * np.sin(b * y[j]) * np.sin(c * z[k])
            u[i, j, k, 1] = B * np.sin(a * x[i]) * np.cos(b * y[j]) * np.sin(c * z[k])
            u[i, j, k, 2] = C * np.sin(a * x[i]) * np.sin(b * y[j]) * np.cos(c * z[k])

In [ ]:
simulator = qiskit_aer.backends.statevector_simulator.StatevectorSimulator()

In [ ]:
# Initialize the Quantum LBM field
Psi_qlbm = np.zeros((TIMESTEPS + 1, N_POINTS_X, N_POINTS_Y, N_POINTS_Z))
Psi_qlbm[0, :, :, :] = Psi_init

# Quantum LBM loop
for t in range(TIMESTEPS):
    u_LBM = u 
    qc = QLBM(density_field=Psi_qlbm[t, :, :, :], velocity_field=u_LBM, number_velocities=NUMBER_DISCRETE_VELOCITIES)
    
    # Compile and run the quantum circuit
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit).result()
    
    # Extract and process the statevector
    statevector = np.array(result.get_statevector())
    real_part_statevector = np.real(statevector[:N_POINTS_X * N_POINTS_Y * N_POINTS_Z])
    reshaped_real_part = np.reshape(real_part_statevector, (N_POINTS_X, N_POINTS_Y, N_POINTS_Z),order='C')
    
    # Normalize and update Psi_qlbm
    norm_factor = np.linalg.norm(Psi_qlbm[t, :, :, :].flatten()) * 4 * np.sqrt(2)
    Psi_qlbm[t + 1, :, :, :] = reshaped_real_part * norm_factor

In [ ]:
# Lattice Boltzmann parameters
LATTICE_VELOCITIES = np.array([
    [0,  1, -1,  0,  0,   0,  0,  1, -1,  1, -1,  0,  0,  1, -1,  1, -1,  0,  0,  1, -1,  1, -1,  1, -1, -1,  1],
    [0,  0,  0,  1, -1,   0,  0,  1, -1,  0,  0,  1, -1, -1,  1,  0,  0,  1, -1,  1, -1,  1, -1, -1,  1,  1, -1],
    [0,  0,  0,  0,  0,   1, -1,  0,  0,  1, -1,  1, -1,  0,  0, -1,  1, -1,  1,  1, -1, -1,  1,  1, -1,  1, -1]
])

LATTICE_WEIGHTS = np.array([
    8/27,                   
    2/27, 2/27, 2/27, 2/27, 2/27, 2/27,  
    1/54, 1/54, 1/54, 1/54, 1/54, 1/54, 1/54, 1/54, 1/54, 1/54, 1/54, 1/54,  
    1/216, 1/216, 1/216, 1/216, 1/216, 1/216, 1/216, 1/216  
])

# Initialize the classical scalar field
Psi_classical = np.zeros((TIMESTEPS + 1, N_POINTS_X, N_POINTS_Y, N_POINTS_Z))
Psi_classical[0, :, :, :] = Psi_init

# Initialize the distribution function
f = np.zeros((27, N_POINTS_X, N_POINTS_Y, N_POINTS_Z))

# Classical Lattice Boltzmann loop
for t in range(TIMESTEPS):
    # Calculate the equilibrium distribution function
    for v in range(27):
        velocity_projection = (
            LATTICE_VELOCITIES[0, v] * u[:, :, :, 0] +
            LATTICE_VELOCITIES[1, v] * u[:, :, :, 1] +
            LATTICE_VELOCITIES[2, v] * u[:, :, :, 2]
        )
        f[v, :, :, :] = LATTICE_WEIGHTS[v] * Psi_classical[t, :, :, :] * (1 + (velocity_projection / c_s**2))

    # Streaming step: Shift each distribution along its velocity direction
    for v in range(27):
        f[v, :, :, :] = np.roll(
            np.roll(
                np.roll(f[v, :, :, :], LATTICE_VELOCITIES[0, v], axis=0),
                LATTICE_VELOCITIES[1, v], axis=1
            ),
            LATTICE_VELOCITIES[2, v], axis=2
        )
    
    # Update the scalar field by summing over all distribution functions
    Psi_classical[t + 1, :, :, :] = np.sum(f, axis=0)

In [ ]:
# Initialize MSE and RMSE arrays
MSE = np.zeros(TIMESTEPS)
RMSE = np.zeros(TIMESTEPS)

# Remove initial state from the classical and quantum fields
Psi_classical_no_init = Psi_classical[1:, :, :, :]
Psi_quantum_no_init = Psi_qlbm[1:, :, :, :]

# Calculate MSE and RMSE for each timestep
for t in range(TIMESTEPS):
    difference = Psi_quantum_no_init[t, :, :, :] - Psi_classical_no_init[t, :, :, :]
    MSE[t] = np.mean(np.square(difference))
    RMSE[t] = np.sqrt(MSE[t])

In [ ]:
plt.figure(figsize=(18.5, 10.5), dpi=100)
plt.plot(range(1, TIMESTEPS + 1), RMSE, linewidth=3, label='D3Q27')
plt.grid()
plt.legend(loc='best')
plt.title('RMSE between Digital and Quantum LBM')
plt.ylabel('Error')
plt.xlabel('Time Step')
plt.show()